In [1]:
import datetime, time, math, os, json, shutil, asyncio, networkx as nx
from utils.gitutils import create_pull_request, clone_repo, create_branch, wipe_repo, prepare_repo
from utils.agent import Agent, GenerationConfig, Interaction, Team
from utils.stringutils import arr_from_sep_string, extract_markdown_blocks, markdown_to_dict, fuzzy_find, skwonk
from utils.filetreeutils import FileTree, write_file_tree
from utils.listutils import flatten
from utils.frameworkutils import DartAnalyzer
from dotenv import load_dotenv
from utils.graphutils import loose_level_order, collapsed_level_order
from utils.frameworkutils import Framework

In [2]:
load_dotenv()  # Load environment variables from .env

GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
API_KEY = os.getenv('API_KEY')

In [3]:
repo_url = 'https://github.com/6165-MSET-CuttleFish/TeamTrack'
source = Framework.FLUTTER
feature = f"""## Feature Name
Spaces
## Descriptions
Spaces are a way for users to share videos of their robot competing. These videos will have information about the team number and name as well. Users will be able to view these videos and see how other teams are doing. They will also be able to upload their own videos to share with others.
"""

In [4]:
# Clone the repo and make a new branch
repo = clone_repo(repo_url)
base_branch = repo.active_branch.name
created_branch = f"modification-{base_branch}"
local_repo_path = str(repo.working_dir)
working_dir_path = f'{local_repo_path}\\{source.get_working_dir()}'

In [5]:
source_swe_summarizer = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_analyzer",
    generation_config=GenerationConfig(temperature=0.7),
    system_prompt=f'''You are a {source} software engineer. Your job is to summarize the functionality of provided code files. Structure your response as follows:

# filename.ext:

$7 sentence summary of the functionality/contents of the file$
'''
)
pm = Agent(
    model_name="gemini-1.5-pro-001",
    api_key=API_KEY,
    name="pm",
    generation_config=GenerationConfig(temperature=0.2),
    system_prompt=f"""You are a high-level technical project manager tasked with the project of adding functionality to a {source} repository."""
)
source_swe_coder = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_coder",
    generation_config=GenerationConfig(temperature=0.5),
    system_prompt=f"You are a {source} software engineer. Your job is to write code to implement a specific feature. Respond with code and nothing else. No explanations needed."
)
source_swe_editor = Agent(
    model_name="gemini-1.5-flash",
    api_key=API_KEY,
    name="source_swe_editor",
    generation_config=GenerationConfig(temperature=0.0),
    system_prompt=f"""You are a {source} software engineer tasked with editing a large code file to implement changes in accordance with the instructions you will receive in the following prompts. Large edits at once are NOT the way. We will solve this problem by dividing the edit into smaller sub-edits consisting of additions and deletions (no more than 10 lines per edit). We will use a custom file type called 'skinnydiff', which are diff-like files but only feature a larger file's substring, to convey these changes. Here is an example of a 'skinnydiff' file:

```skinnydiff
 void foo() {{
- int x = 5;
+ var x = 7;
  return x + 3; // Example code 
 }}
```
Skinnydiffs are shorter versions of diff files that only include a few context lines around the deletions and insertions.

Given this knowledge, structure your response as follows:

# Step 1
$Talk through the 1st sub-change and what you plan to do$
## Scope
$Brief planning on which line or class to include in the change$
## Change 1
```skinnydiff
$code diff$
```

# Step N
$Talk through the Nth sub-change and what you plan to do$
## Scope
$Brief planning on which line or class to include in the change$
## Change N
```skinnydiff
$...$
```"""
)
skeletonizer = Agent(
    name="skeletonizer",
    model_name="gemini-1.5-flash",
    api_key=API_KEY,
    system_prompt="""Why say many word when few word do trick?! Turn code into skeleton-pseudocode by condensing nitty gritty syntax into sensible short pseudocode. Respond as such: 
```pseudocode
....
```""",
    generation_config=GenerationConfig(temperature=0.0)
)

team = Team(source_swe_summarizer, pm, source_swe_coder, source_swe_editor, skeletonizer)

In [6]:
analyzer = source.get_analyzer(working_dir_path)

source_dependency_graph = analyzer.buildDependencyGraph()

In [9]:
# loose level order
eval_order = loose_level_order(source_dependency_graph)[::-1]
for level in eval_order:
    print(level)
print(f"Number of levels: {len(eval_order)}")

['models\\StatConfig.dart', 'models\\AppModel.dart']
['models\\GameModel.dart']
['models\\ScoreModel.dart']
['functions\\Extensions.dart', 'components\\misc\\PlatformGraphics.dart']
['components\\statistics\\BarGraph.dart', 'components\\users\\PFP.dart']
['functions\\Statistics.dart', 'components\\scores\\Incrementor.dart', 'components\\scores\\ScoreSummary.dart', 'components\\users\\UsersRow.dart', 'providers\\Auth.dart']
['views\\home\\match\\MatchView.dart']
['components\\statistics\\PercentChange.dart', 'functions\\Functions.dart']
['components\\scores\\ScoreTimeline.dart', 'models\\Change.dart', 'components\\scores\\ScoreRangeSummary.dart']
['views\\home\\change\\ChangeConfig.dart', 'views\\home\\match\\MatchConfig.dart', 'components\\misc\\CardView.dart', 'components\\scores\\ScoringElementStats.dart', 'views\\home\\change\\ChangeRow.dart', 'views\\home\\match\\ExampleMatchRow.dart', 'views\\home\\match\\MatchRow.dart', 'components\\statistics\\CheckList.dart', 'components\\misc\

In [10]:
(ranks := sorted(nx.pagerank(source_dependency_graph).items(), key=lambda x: x[1], reverse=True))

[('functions\\Functions.dart', 0.13015165084511265),
 ('models\\AppModel.dart', 0.1133542274603946),
 ('models\\GameModel.dart', 0.11054238484593704),
 ('components\\misc\\PlatformGraphics.dart', 0.07280617686501588),
 ('functions\\Extensions.dart', 0.06491036089133999),
 ('models\\ScoreModel.dart', 0.04846332476566593),
 ('functions\\Statistics.dart', 0.04758879390256111),
 ('models\\StatConfig.dart', 0.03794723616207956),
 ('views\\home\\match\\MatchView.dart', 0.03251008281339123),
 ('models\\Change.dart', 0.02251042215978528),
 ('providers\\Auth.dart', 0.01548168224778635),
 ('components\\statistics\\BarGraph.dart', 0.012206693129943554),
 ('components\\users\\PFP.dart', 0.009606083157777829),
 ('models\\GPTModel.dart', 0.009186070453302648),
 ('views\\LandingPage.dart', 0.008410495014922857),
 ('components\\scores\\ScoreSummary.dart', 0.007612721724091385),
 ('components\\users\\UsersRow.dart', 0.007612721724091385),
 ('components\\misc\\EmptyList.dart', 0.0075104656743458665),
 (

In [11]:
max_probability = ranks[0][1]
(percentage_of_max_arr := [(rank[0], rank[1] / max_probability) for rank in ranks])

[('functions\\Functions.dart', 1.0),
 ('models\\AppModel.dart', 0.8709396056396712),
 ('models\\GameModel.dart', 0.8493352495197185),
 ('components\\misc\\PlatformGraphics.dart', 0.5593949549795498),
 ('functions\\Extensions.dart', 0.4987286789668673),
 ('models\\ScoreModel.dart', 0.3723604307050999),
 ('functions\\Statistics.dart', 0.3656411086110179),
 ('models\\StatConfig.dart', 0.29156169680274574),
 ('views\\home\\match\\MatchView.dart', 0.24978617330086691),
 ('models\\Change.dart', 0.17295533336395305),
 ('providers\\Auth.dart', 0.11895110163612424),
 ('components\\statistics\\BarGraph.dart', 0.09378823127238059),
 ('components\\users\\PFP.dart', 0.07380684836037597),
 ('models\\GPTModel.dart', 0.0705797459629195),
 ('views\\LandingPage.dart', 0.06462073250943079),
 ('components\\scores\\ScoreSummary.dart', 0.05849116530339616),
 ('components\\users\\UsersRow.dart', 0.05849116530339616),
 ('components\\misc\\EmptyList.dart', 0.05770549682296168),
 ('components\\scores\\Increment

In [12]:
(source_file_tree := FileTree.from_dir(working_dir_path))

├── api\
│   ├── README.MD
├── components\
│   ├── misc\
│   │   ├── CardView.dart
│   │   ├── Collapsible.dart
│   │   ├── EmptyList.dart
│   │   ├── InfoPills.dart
│   │   ├── PlatformGraphics.dart
│   ├── scores\
│   │   ├── Incrementor.dart
│   │   ├── ScoreCard.dart
│   │   ├── ScoreRangeSummary.dart
│   │   ├── ScoreSummary.dart
│   │   ├── ScoreTimeline.dart
│   │   ├── ScoringElementStats.dart
│   ├── statistics\
│   │   ├── BarGraph.dart
│   │   ├── CheckList.dart
│   │   ├── PercentChange.dart
│   ├── users\
│   │   ├── PFP.dart
│   │   ├── UsersRow.dart
│   ├── AutonomousDrawingTool.dart
├── functions\
│   ├── APIMethods.dart
│   ├── Extensions.dart
│   ├── Functions.dart
│   ├── GPTModel.dart
│   ├── ResponseModel.dart
│   ├── Statistics.dart
├── models\
│   ├── AppModel.dart
│   ├── Change.dart
│   ├── GameModel.dart
│   ├── GPTModel.dart
│   ├── ScoreModel.dart
│   ├── StatConfig.dart
├── providers\
│   ├── Auth.dart
│   ├── PushNotifications.dart
│   ├── Theme.dart
│   ├

In [27]:
async def skeletonize(node):
    name = source_file_tree.nodes[node]['name']
    content = source_file_tree.nodes[node]['content']
    message = f"{name}\n```\n{content}\n```"
    return await team.async_chat_with_agent(
        agent_name='skeletonizer',
        message=message,
        save_keys=[f"skeleton_{node}", "all_skeletons"],
        prompt_title=f"Skeleton of {node}"
    )
order = source_file_tree.get_files()
tasks = [skeletonize(node) for node in order]
skeletons = await asyncio.gather(*tasks)
for i, response in enumerate(skeletons):
    source_file_tree.nodes[order[i]]['skeleton'] = response

In [28]:
# Summarize the original repo
async def summarize(node):
    name = source_file_tree.nodes[node]['name']
    content = source_file_tree.nodes[node]['content']
    message = f"{name}\n```\n{content}\n```"
    return await team.async_chat_with_agent(
        agent_name='source_swe_analyzer',
        message=message,
        context_keys=[f"skeleton_{neighbor}" for neighbor in source_dependency_graph[node]],
        save_keys=[f"summary_{node}", "all_summaries"],
        prompt_title=f"Summary of {node}"
    )
tasks = [summarize(node) for node in order]
responses = await asyncio.gather(*tasks)
for i, response in enumerate(responses):
    source_file_tree.nodes[order[i]]["summary"] = response

In [29]:
prompt = f"""This is the file tree for the original {source} repo:
```
{source.get_working_dir()}\\
{source_file_tree}
```
Given the prior context, summarize the functionality of the entire repository succinctly.
"""
print(response := team.chat_with_agent('source_swe_analyzer', prompt, context_keys=["all_summaries"], save_keys=["all_summaries"], prompt_title=f"""Repository Summary.
File Tree provided:
```
{source.get_working_dir()}\\
{source_file_tree}
```
"""))

The repository implements a Flutter application called TeamTrack, designed to manage and analyze data for FIRST Robotics Competition (FRC) events. The app allows users to:

- **Create and manage events:** Create both local and remote events, import match schedules from images, and fetch scores from The Orange Alliance API.
- **Manage teams:** View team information, add and delete teams, sort teams by different statistics, and view team-specific scores and charts.
- **Manage matches:** View and edit match scores, add new matches, and track match progress with a timer.
- **Analyze team performance:** Utilize various statistical tools to analyze team scores, including mean, median, best, and standard deviation. View score timelines and detailed scoring element statistics.
- **Simulate alliances:** Explore potential alliance combinations using a simulator that provides AI-generated recommendations based on team performance.
- **Draw autonomous paths:** Use a drawing tool to visualize auton

## Summary Compression

In [30]:
curr_string = '\n'.join([message['parts'][0] for interaction in team.context_threads['all_summaries'] for message in interaction.to_dict()])
(curr_tokens := pm.model.count_tokens(curr_string).total_tokens)

17093

In [31]:
would_be_string = '\n'.join([source_file_tree.nodes[node]['content'] for node in source_dependency_graph.nodes])
(would_be_tokens := pm.model.count_tokens(would_be_string).total_tokens)

122951

In [32]:
(percent_tokens_reduction := 1 - (curr_tokens / would_be_tokens))

0.8609771372335321

## Skeleton Compression

In [33]:
curr_string = '\n'.join([message['parts'][0] for interaction in team.context_threads['all_skeletons'] for message in interaction.to_dict()])
(curr_tokens := pm.model.count_tokens(curr_string).total_tokens)

30267

In [34]:
would_be_string = '\n'.join([source_file_tree.nodes[node]['content'] for node in source_dependency_graph.nodes])
(would_be_tokens := pm.model.count_tokens(would_be_string).total_tokens)

122951

In [35]:
(percent_tokens_reduction := 1 - (curr_tokens / would_be_tokens))

0.7538287610511505

In [54]:
response_format = f"""\
WRITES must include: the path to the new file, the path to necessary local dependencies (N/A if no local dependencies) needed to build the new file, and the high level overview of what to do to build the new file.
EDITS must include: the path to the file, the path to local dependencies (N/A if no local dependencies) needed to edit the file, and the high level overview of what to do to edit the file.
DELETES must include: the path to the file, a short deletion message, and internal reasons for deletion.

Structure your response using the following example:
```
| WRITE | {source.get_working_dir()}\\path\\to\\newfile.ext | {source.get_working_dir()}\\path\\to\\dependency1, {source.get_working_dir()}\\path\\to\\dependency2 (N/A if no dependencies) | Implement $(feature)$ with the following properties:
    - property1
    - property2
| EDIT | {source.get_working_dir()}\\path\\to\\file.ext | {source.get_working_dir()}\\path\\to\\dependency1, {source.get_working_dir()}\\path\\to\\dependency2 (N/A if no new dependencies) | Add $(feature)$ by doing:
    - step1
    - step2
| DELETE | {source.get_working_dir()}\\path\\to\\some\\file.ext | Short deletion message | Internal reasons for deletion:
    - reason1
    - reason2
```
Respond ONLY with the high level overview of what to do. The actual implementation will be done by a coder.\
"""

# Create new file tree
prompt = f'''\
You are to build the following feature:
{feature}
------------------
{response_format}
------------------
You need to follow the response template structure exactly (MY JOB DEPENDS ON IT)\
'''
print(response := team.chat_with_agent("pm", prompt, context_keys=["all_summaries"]))

```
| WRITE | lib/views/spaces/SpacesList.dart | lib/models/SpaceModel.dart, lib/components/video_player.dart | Implement SpacesList with the following properties:
    - Displays a list of Space cards.
    - Each card shows a preview of the video, team number, and team name.
    - Allows navigation to SpaceDetails view for a selected space.
| WRITE | lib/views/spaces/SpaceDetails.dart | lib/models/SpaceModel.dart, lib/components/video_player.dart | Implement SpaceDetails with the following properties:
    - Displays the full video using a video player.
    - Shows details like team number, team name, and upload date.
| WRITE | lib/views/spaces/UploadSpace.dart | lib/models/SpaceModel.dart, lib/services/firebase_storage.dart | Implement UploadSpace with the following properties:
    - Allows users to select a video from their device.
    - Takes input for team number and team name.
    - Uploads the video and its metadata to Firebase Storage.
| WRITE | lib/models/SpaceModel.dart | N/A |

In [56]:
md = extract_markdown_blocks(response)[0]
blocks = md.split("|")
actions = []
i = 0
while i < len(blocks):
    block = blocks[i].strip()
    if block != "WRITE" and block != "EDIT" and block != "DELETE":
        i += 1
        continue
    action, file, dependencies, prompt = blocks[i:i + 4]
    action = action.strip()
    if file.strip().startswith(source.get_working_dir()):
        file = file.strip()[len(source.get_working_dir()) + 1:]
    else:
        file = file.strip()
    if dependencies.strip() == "N/A" or dependencies.strip() == "":
        dependencies = []
    else:
        dependencies = [source_file_tree.get_closest_file_name(d) for d in dependencies.split(",")]
    prompt = prompt.strip()
    actions.append((action, file, dependencies, prompt))
    i += 4
for action in actions:
    print(action)

('WRITE', 'views/spaces/SpacesList.dart', ['models\\GameModel.dart', 'components\\misc\\CardView.dart'], 'Implement SpacesList with the following properties:\n    - Displays a list of Space cards.\n    - Each card shows a preview of the video, team number, and team name.\n    - Allows navigation to SpaceDetails view for a selected space.')
('WRITE', 'views/spaces/SpaceDetails.dart', ['models\\GameModel.dart', 'components\\misc\\CardView.dart'], 'Implement SpaceDetails with the following properties:\n    - Displays the full video using a video player.\n    - Shows details like team number, team name, and upload date.')
('WRITE', 'views/spaces/UploadSpace.dart', ['models\\GameModel.dart', 'providers\\UserPresence.dart'], 'Implement UploadSpace with the following properties:\n    - Allows users to select a video from their device.\n    - Takes input for team number and team name.\n    - Uploads the video and its metadata to Firebase Storage.')
('WRITE', 'models/SpaceModel.dart', [], 'Impl

In [57]:
import re
async def write(file_path, dependencies, prompt):
    response = await team.async_chat_with_agent(
        "source_swe_coder",
        f"Dependencies: {dependencies}\nInstructions for {file_path}:\n{prompt}",
        context_keys=[f"skeleton_{d}" for d in dependencies]
    )
    md = extract_markdown_blocks(response)[0]
    path = f"{working_dir_path}\\{file_path}"
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    with open(path, 'w') as f:
        f.write(md)

async def edit(file_path, dependencies, prompt):
    with open(f"{working_dir_path}\\{file_path}", 'r') as f:
        content = f.read()
    response = await source_swe_editor.async_chat(
        f"{prompt}\n\nRemember, try to keep each hunk short and concise.",
        custom_context=flatten([team.context_threads[f"skeleton_{d}"] for d in dependencies]) + [Interaction(f"Contents of {file_path}", content)]
    )
    diffs = extract_markdown_blocks(response)
    for diff in diffs:
        with open(f'{working_dir_path}\\{file_path}', 'w') as f:
            f.seek(0)
            f.truncate()
            content = skwonk(content, diff)
            f.write(content)

def delete(file_path):
    os.remove(f"{working_dir_path}\\{file_path}")

In [58]:
from utils.errorutils import repeat_until_finish
from tqdm import tqdm

for action, file, dependencies, prompt in tqdm(actions):
    if action == "WRITE":
        await write(file, dependencies, prompt)
    elif action == "EDIT":
        await edit(file, dependencies, prompt)
    elif action == "DELETE":
        delete(file)


 17%|█▋        | 1/6 [00:03<00:19,  3.96s/it]

index: 0
finish_reason: RECITATION



100%|██████████| 6/6 [00:37<00:00,  6.20s/it]
